In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
comment_df = pd.read_excel("..\data\cleaned_swed_lemma.xlsx",
                            )
comment_df.drop(index = [9, 1555, 2474, 2483, 4346, 7212], inplace = True)

#remove short words
comment_df.text = comment_df.text.map(lambda line: " ".join( [word for word in line.split(" ") if (len(word) > 3 or word == "mzp")] ))

In [37]:
comment_df[comment_df.author.str.contains("bot", case = False)]

,Unnamed: 0,Unnamed: 0.1,comment_id,text,author,upvotes,timestamp,post_id,lens
880,880,1726,hg8o3ns,looks like posted link these should load faste...,AmputatorBot,9,1633967331,q5yryi,49
952,952,1877,hg88pze,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,2,1633960547,q5we21,22
1020,1020,1993,hg7u2v8,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,1,1633952137,q5tx58,22
1435,1435,2777,hg09axq,looks like posted link these should load faste...,AmputatorBot,2,1633802545,q4qiny,61
1633,1633,3104,hfwc6gt,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,1,1633725082,q465u6,22
2059,2059,3999,hfrc1ys,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,1,1633631250,q3fcxo,22
2210,2210,4362,hfpt4wn,ovodásoknak érik propaganda járványügyi oldal ...,ShampooBottle493,12,1633606327,q36gdf,37
2513,2513,4883,hfm1j71,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,1,1633532973,q2mfd0,22
3110,3110,6001,hfa0ae4,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,1,1633297636,q0rlby,22
3201,3201,6180,hf81gqc,androidon úszni reklámot vanced firefoxban ugy...,oliviaisarobot,10,1633267430,q0hgvr,146


In [51]:


#remove bot text
comment_df.drop(index = comment_df[comment_df.author.str.contains("bot", case = False)].index, inplace = True)

tf_idf_sk = TfidfVectorizer(norm="l1")
X = tf_idf_sk.fit_transform(comment_df.text)


In [58]:
from sklearn.decomposition import LatentDirichletAllocation

n_comp = 40

lda = lda = LatentDirichletAllocation(n_components=n_comp, random_state=32)
lda.fit(X)


LatentDirichletAllocation(n_components=40, random_state=32)

In [59]:
encoding_matrix2 = pd.DataFrame(lda.components_, 
                               index = [f"topic{i}" for i in range(n_comp)], 
                               columns = tf_idf_sk.get_feature_names())

top_by_topic2 = pd.DataFrame()
for col in encoding_matrix2.index:
    topic_temp_df2 = pd.DataFrame(encoding_matrix2.T.sort_values(col, axis = "rows", ascending = False, 
                                                               key = np.abs)[col][:5]).reset_index()
    topic_temp_df2.columns = [col, col+"_value"]
    top_by_topic2 = pd.concat([top_by_topic2, topic_temp_df2], axis = 1)

C:\Users\balin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [61]:
encoding_matrix2

,30,aaaaa,aaaaaa,aaaaaaaaaa,aaaaaaaaaaah,aacgtatgggcctatatttgcccgacatagggcggcctaattgattgacatagcgtatgggcctatgacataggcatagggcttatgaccaaggttaag,aacgtatgggcctatatttgcccgacataggggggcctaattgattgacatagcgtatgggcctatgacataggcatagggcttatgaccaaggttaag,abbahagy,abbahagytaa,abbol,...,őszinteségre,őszintétlennek,őszöd,őszödi,őzbak,őélkasdjfasj,őőőő,űrpufajkás,űrítése,űzi
topic0,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,...,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000
topic1,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.173368,...,0.025000,0.025000,0.025000,0.025000,0.206959,0.025000,0.025000,0.025000,0.025000,0.025000
topic2,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,...,0.025000,0.044877,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000
topic3,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.125633,0.025000,0.025000,...,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000
topic4,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.054006,...,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000
topic5,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,...,0.025000,0.025000,0.025000,0.025000,0.025000,0.046212,0.025000,0.025000,0.025000,0.025000
topic6,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,...,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000
topic7,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.057537,...,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000
topic8,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,...,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.145126,0.025000,0.025000
topic9,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,...,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.050711


In [47]:
top_by_topic2

,topic0,topic0_value,topic1,topic1_value,topic2,topic2_value,topic3,topic3_value
0,mzp,39.201768,that,1.642221,poll,2.424474,like,2.412671
1,szavaz,35.643128,ingyen,1.209268,peter,1.696897,goes,1.332015
2,fidesz,32.873602,nice,1.182397,still,1.346392,elso,1.088088
3,dobrev,28.584261,with,1.147957,here,1.301450,they,1.060481
4,karácsony,26.701343,miert,1.141069,gyerekeid,1.249108,tell,1.003251
5,mond,25.937467,gyula,0.976485,bernie,1.129916,címe,0.982410
6,ember,25.255229,count,0.949605,erzsébet,0.982186,gyurcsany,0.937505
7,akar,22.329264,people,0.940785,boldogság,0.819406,christmas,0.893043
8,orbán,21.132516,szemelyesen,0.909479,erdekel,0.810534,with,0.862083
9,megy,20.907735,news,0.900048,elott,0.808146,kari,0.824254
